In [4]:
import pandas as pd
import numpy as np
import os

In [5]:
dataDir = '../data'

In [6]:
user_df = pd.read_csv(os.path.join(dataDir, 'member_info_1020.csv'),
                              #names= ['user_id', 'gender', 'visit_freq', 'binary_A',
                               #      'binary_B', 'binary_C', 'binary_D', 'binary_E',
                                #     'category_A' 'category_B', 'category_C',
                                 #    'category_D', 'category_E', 'salt', 'follow_topics', 'interest_topics'],
                              #sep= ',',
                      index_col= 'user_id',
                      # nrows= 5000
                     )

In [7]:
user_df.head()

,gender,visit_freq,binary_A,binary_B,binary_C,binary_D,binary_E,category_A,category_B,category_C,category_D,category_E,salt_value,follow_topics,interest_topics
user_id,,,,,,,,,,,,,,,
M1934753188,male,monthly,0,1,0,1,0,MD470265,BR470265,PV929066,CT929066,PF470265,764,T540,"T21107:1.7915097,T405:1.6123838,T4436:1.518003..."
M595924114,male,daily,0,0,0,1,1,Others,BR596936,PV002320,CT840234,PF470265,671,"T44126,T15940,T839,T8978,T2934,T1113,T3914,T12...","T18016:2.0650618,T2384:1.2503042,T1142:1.13569..."
M1473482940,female,weekly,0,1,0,1,0,MD116493,BR641329,PV170953,CT470265,PF470265,454,"T30874,T2113,T8656,T21,T523,T8,T116,T5727,T68,...","T46:1.330939,T2159:1.1296458,T379:1.1241927,T1..."
M578477092,male,daily,1,1,0,1,0,MD889589,BR803759,PV545833,CT545833,PF470265,588,"T946,T7323,T297,T2660,T36067,T53107,T2654,T507...","T15918:1.9479566,T8106:1.8578106,T4787:1.58486..."
M1088794709,male,weekly,0,1,0,0,0,MD825760,BR641329,PV071037,CT470265,PF470265,361,"T582,T558,T28776,T5186,T9081,T2419,T2693,T2299...","T65:1.5992582,T867:1.3179373,T93:1.2095009,T31..."


In [16]:
user_df.describe()

,binary_A,binary_B,binary_C,binary_D,binary_E,salt_value,follow_topics_mp
count,1.931654e+06,1.931654e+06,1.931654e+06,1.931654e+06,1.931654e+06,1.931654e+06,1.931654e+06
mean,5.973637e-01,3.117147e-01,2.953117e-02,1.475352e-01,5.420381e-02,3.165209e-01,9.658265e+05
std,4.904288e-01,4.631941e-01,1.692900e-01,3.546388e-01,2.264195e-01,1.039011e-01,5.576206e+05
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.515723e-01,4.829132e+05
50%,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.830189e-01,9.658265e+05
75%,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.496855e-01,1.448740e+06
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.931653e+06


### topic word embedding

In [4]:
topic_ids = pd.read_csv(os.path.join(dataDir, 'topic_vectors_64d.txt'),
                        names = ['topic_id', 'vector'],
                        usecols = ['topic_id', ],
                        index_col= 'topic_id',
                        sep = '\t',
                        squeeze= True)

In [53]:
num_topics = len(topic_ids)

In [96]:
topic_vec = np.load(os.path.join(dataDir, 'topic_array.npy'))

In [97]:
topic_vec.dtype

dtype('float64')

## 处理　follow topics:

In [92]:
topic_idmap = pd.Series(np.int64(np.arange(len(topic_ids))), index = topic_ids.index)

In [93]:
topic_idmap['-1'] = 10000

In [94]:
type(topic_idmap)

pandas.core.series.Series

In [95]:
topic_idmap.dtype

dtype('int64')

#### -1 对应的　topic 以加权均值填充

In [99]:
tp_count = np.zeros((num_topics, 1))
for tp_s in user_df['follow_topics']:
    if tp_s != '-1':
        id_list = s.split(',')
        ind_list = topic_idmap[id_list].tolist()
        for ind in ind_list:
            tp_count[ind] += 1

In [100]:
tp_weighted_mean = np.max(np.multiply(topic_vec, tp_count), axis= 0) / np.sum(tp_count)

In [101]:
tp_weighted_mean.shape

(64,)

In [102]:
topic_vec_cat = np.concatenate([topic_vec, tp_weighted_mean.reshape(1, -1)], axis= 0)

In [103]:
topic_vec_cat.shape

(100001, 64)

In [104]:
def map_topics(s):
    try:
        id_list = s.split(',')
        ind_list = topic_idmap[id_list].tolist()
        topic_array = topic_vec[ind_list]
        topic_mp = np.max(topic_array, axis= 0)
    except:
        print(ind_list)
    return topic_mp

In [105]:
user_df['follow_topics_mp_vec'] = user_df['follow_topics'].apply(map_topics)

In [106]:
user_df['follow_topics_mp_vec'].head()

user_id
M1934753188    [6.125305, -1.4180568, -1.3143845, -2.9268239,...
M595924114     [3.8494484, 3.977306, 5.7516003, 4.9546337, 4....
M1473482940    [3.4723787, 4.6616564, 3.5893476, 5.427303, 5....
M578477092     [8.753078, 7.646762, 7.8675485, 5.283248, 6.17...
M1088794709    [9.542506, 4.7182026, 4.145554, 2.87201, 7.516...
Name: follow_topics_mp_vec, dtype: object

In [112]:
follow_tp_mp_array = np.array(user_df['follow_topics_mp_vec'].tolist())

In [113]:
follow_tp_mp_array.shape

(1931654, 64)

In [152]:
np.save(os.path.join(dataDir, 'user_follow_topics_mp.npy'), follow_tp_mp_array)

### 添加topic 对应索引列

In [8]:
user_df['follow_topics_mp'] = np.arange(len(user_df))

### 盐值 salt value归一化

In [9]:
from sklearn.preprocessing import MinMaxScaler

In [10]:
saltValueScaler = MinMaxScaler((0, 1))

In [12]:
user_df['salt_value'] = saltValueScaler.fit_transform(user_df[['salt_value', ]])

/home/sc3/miniconda2/envs/ydzhang/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


### 编码类别属性

In [15]:
from sklearn.preprocessing import LabelEncoder

In [17]:
gender_encoder = LabelEncoder()

In [18]:
user_df['gender'] = gender_encoder.fit_transform(user_df['gender'])

In [19]:
visitFreq_encoder = LabelEncoder()

In [21]:
user_df['visit_freq'] = visitFreq_encoder.fit_transform(user_df['visit_freq'])

In [22]:
category_A_encoder = LabelEncoder()

user_df['category_A'] = category_A_encoder.fit_transform(user_df['category_A'])

In [24]:
category_B_encoder = LabelEncoder()

user_df['category_B'] = category_B_encoder.fit_transform(user_df['category_B'])

In [25]:
category_C_encoder = LabelEncoder()

user_df['category_C'] = category_C_encoder.fit_transform(user_df['category_C'])

In [28]:
category_D_encoder = LabelEncoder()

user_df['category_D'] = category_D_encoder.fit_transform(user_df['category_D'])

In [27]:
category_E_encoder = LabelEncoder()

user_df['category_E'] = category_E_encoder.fit_transform(user_df['category_E'])

### 输出user_df

In [30]:
user_df.to_csv(os.path.join(dataDir, 'member_info_1021.csv'),
                              columns= ['gender', 'visit_freq', 'binary_A',
                                    'binary_B', 'binary_C', 'binary_D', 'binary_E',
                                    'category_A', 'category_B', 'category_C',
                                    'category_D', 'category_E', 'salt_value', 'follow_topics_mp', 'interest_topics'],
                              sep= '\t',)

In [149]:
help(pd.DataFrame.to_csv)

Help on function to_csv in module pandas.core.generic:

to_csv(self, path_or_buf=None, sep=',', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, mode='w', encoding=None, compression='infer', quoting=None, quotechar='"', line_terminator=None, chunksize=None, tupleize_cols=None, date_format=None, doublequote=True, escapechar=None, decimal='.')
    Write object to a comma-separated values (csv) file.
    
    .. versionchanged:: 0.24.0
        The order of arguments for Series was changed.
    
    Parameters
    ----------
    path_or_buf : str or file handle, default None
        File path or object, if None is provided the result is returned as
        a string.  If a file object is passed it should be opened with
        `newline=''`, disabling universal newlines.
    
        .. versionchanged:: 0.24.0
    
           Was previously named "path" for Series.
    
    sep : str, default ','
        String of length 1. Field delimiter for the output

### TODO: 处理　interest_topic

In [131]:
tp_weight = np.zeros((num_topics, 1))
for tp_s in user_df['interest_topics']:
    if tp_s != '-1':
        id_str_list = tp_s.split(',')
        for id_str in id_str_list:
            tp_id, weight_str = id_str.split(':')
            weight = float(weight_str)
            # print(tp_id, weight)
            tp_weight[topic_idmap[tp_id]] += weight

In [134]:
tp_weight[:100,]

array([[    0.        ],
       [51948.39888056],
       [33987.26315429],
       [17684.07810108],
       [40966.03172979],
       [22649.40822608],
       [15049.13911903],
       [40835.19418761],
       [48815.12520887],
       [20187.81928158],
       [23216.35223568],
       [13801.42462087],
       [17182.09323776],
       [ 5656.35933362],
       [15851.46695401],
       [27580.26397775],
       [29635.54168217],
       [41146.43601064],
       [34605.12741778],
       [14411.78559699],
       [10410.12467941],
       [29863.90690965],
       [20313.63107364],
       [20513.03201613],
       [20674.17432297],
       [34228.18583081],
       [10657.10852358],
       [32626.13109777],
       [30295.55705368],
       [ 4933.24136651],
       [67045.66956224],
       [17145.51438338],
       [ 8797.63776966],
       [ 8251.64858598],
       [12180.62890914],
       [23916.00224894],
       [21275.02661513],
       [24758.36588506],
       [19960.98524567],
       [16299.62462713],


In [140]:
np.any(np.isnan(tp_weight))

False

In [141]:
np.any(np.isnan(topic_vec))

False

In [142]:
topic_vec.shape

(100000, 64)

In [143]:
tp_weight.shape

(100000, 1)

In [145]:
tp_weight.dtype

dtype('float64')

In [148]:
np.sum(tp_weight/10000)

inf

In [137]:
int_tp_weighted_mean = np.dot(tp_weight.reshape(1, -1) / np.sum(tp_weight), topic_vec)

/home/sc3/miniconda2/envs/ydzhang/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [138]:
int_tp_weighted_mean

array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]])

In [ ]:
def map_topics(s):
    try:
        if tp_s != '-1':
            id_str_list = tp_s.split(',')
            tp_list = []
            weight_list = []
            for id_str in id_str_list:
                tp_id, weight_str = id_str.split(':')
                weight = float(weight_str)
                tp_list.append(tp_id)
                weight_list.append(weight)
            tp_inds = tp_idmap[tp_list]
            tp_array = tp_vec[tp_inds]
            weight_array = np.array(weight_list).reshape(1, -1)
            weighted_tp = np.dot(weight_array, tp_array)
        else:
            weighted_tp = 
    except:
        print(ind_list)
    return topic_mp

### question topics max pooling

In [157]:
quest_df = pd.read_csv(os.path.join(dataDir, 'question_info_1020.csv'),
                       usecols = ['question_id', 'title_SW', 'title_W', 'question_topics', 'create_day'],
                                    index_col= 'question_id')

In [159]:
len(quest_df)

1829900

In [158]:
np.sum(quest_df['question_topics'] == '-1')

3874

In [160]:
topics_mean = np.mean(topic_vec, axis= 0)

In [161]:
topics_mean.shape

(64,)

In [164]:
def map_quest_topics(s):
    try:
        if s != '-1':
            id_list = s.split(',')
            ind_list = topic_idmap[id_list].tolist()
            topic_array = topic_vec[ind_list]
            topic_mp = np.max(topic_array, axis= 0)
        else:
            topic_mp = topics_mean
    except:
        print(ind_list)
    return topic_mp

In [165]:
quest_topics_mp = quest_df['question_topics'].map(map_quest_topics)

In [166]:
quest_topics_mp.head()

question_id
Q2234111670    [2.028344, 4.240323, 1.5969335, -1.7983502, 3....
Q760329790     [-1.8734853, -4.4524074, -1.4084837, -0.096429...
Q741313548     [1.1679975, -0.9902606, 2.8614578, 2.517082, -...
Q3481466230    [-0.8644146, -0.91238177, 1.4963309, 4.5643163...
Q3966197028    [5.1905546, 4.887342, 0.9944194, 6.2047296, 1....
Name: question_topics, dtype: object

In [167]:
quest_topics_mp_array = np.array(quest_topics_mp.tolist())

In [168]:
quest_df['question_topics_mp'] = np.arange(len(quest_df))

In [172]:
np.save(os.path.join(dataDir, 'question_topics_mp.npy'), quest_topics_mp_array)

In [173]:
quest_df.to_csv(os.path.join(dataDir, 'question_info_1021.csv'),
               columns = ['question_id', 'title_SW', 'title_W', 'question_topics_mp', 'create_day'],)

/home/sc3/miniconda2/envs/ydzhang/lib/python3.6/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


### answer topics 处理：

In [180]:
answer_df = pd.read_csv(os.path.join(dataDir, 'answer_info_1021.csv'),
                                #names = ['answer_id', 'question_id', 'user_id', 'create_time',
                                #         'ans_SW', 'ans_W', 'is_good', 'has_picture', 'has_video',
                                #         'word_count', 'num_zan', 'num_cancel', 'num_comment', 'num_collect',
                                #         'num_thanks', 'num_report', 'num_useless', 'num_oppos'],
                                index_col= 'answer_id')

In [181]:
answer_df.head()

,question_id,user_id,create_day,create_hour,answer_SW,answer_W,is_good,has_picture,has_video,word_count,num_zan,num_cancel_zan,num_comment,num_collect,num_thanks,num_report,num_useless,num_oppose
answer_id,,,,,,,,,,,,,,,,,,
A2502060945,Q1867533817,M625498202,3808,7,"SW13,SW19,SW44,SW150,SW23,SW594,SW1254,SW91,SW...","W239,W10528,W142,W20372,W6473,W10,W24,W4527,W2...",0,0,0,41,1,0,1,0,1,0,0,0
A2847829478,Q3366788616,M142330444,3810,17,"SW898,SW3656,SW2,SW413,SW601,SW2,SW2541,SW681,...","W4628,W66060,W1607,W2647,W53385,W109029,W319,W...",0,0,0,204,1,0,0,3,1,0,0,0
A2005999231,Q4264694221,M771499642,3853,13,"SW9,SW510,SW573,SW1348,SW1304,SW326,SW509,SW22...","W3532,W69108,W12395,W81311,W498,W850,W107789,W...",0,0,0,54,2,0,1,0,0,0,0,0
A14821523,Q1088851650,M2282072267,3859,18,"SW7,SW16,SW5,SW144,SW302,SW20,SW1752,SW60,SW75...","W1425,W1676,W3700,W325,W6485,W1090,W8342,W9689...",0,0,0,42,1,0,1,0,1,0,0,0
A731550034,Q1023877868,M2282072267,3855,22,"SW7,SW15,SW596,SW407,SW1698,SW68,SW2,SW56,SW59...","W23420,W2610,W5506,W1489,W8749,W504,W1718,W252...",0,0,0,44,0,0,0,0,0,0,0,0


In [182]:
answer_df['question_topics_mp'] = quest_df.loc[answer_df['question_id'], 'question_topics_mp'].values

In [183]:
answer_df.head()

,question_id,user_id,create_day,create_hour,answer_SW,answer_W,is_good,has_picture,has_video,word_count,num_zan,num_cancel_zan,num_comment,num_collect,num_thanks,num_report,num_useless,num_oppose,question_topics_mp
answer_id,,,,,,,,,,,,,,,,,,,
A2502060945,Q1867533817,M625498202,3808,7,"SW13,SW19,SW44,SW150,SW23,SW594,SW1254,SW91,SW...","W239,W10528,W142,W20372,W6473,W10,W24,W4527,W2...",0,0,0,41,1,0,1,0,1,0,0,0,350889
A2847829478,Q3366788616,M142330444,3810,17,"SW898,SW3656,SW2,SW413,SW601,SW2,SW2541,SW681,...","W4628,W66060,W1607,W2647,W53385,W109029,W319,W...",0,0,0,204,1,0,0,3,1,0,0,0,1691739
A2005999231,Q4264694221,M771499642,3853,13,"SW9,SW510,SW573,SW1348,SW1304,SW326,SW509,SW22...","W3532,W69108,W12395,W81311,W498,W850,W107789,W...",0,0,0,54,2,0,1,0,0,0,0,0,525674
A14821523,Q1088851650,M2282072267,3859,18,"SW7,SW16,SW5,SW144,SW302,SW20,SW1752,SW60,SW75...","W1425,W1676,W3700,W325,W6485,W1090,W8342,W9689...",0,0,0,42,1,0,1,0,1,0,0,0,304256
A731550034,Q1023877868,M2282072267,3855,22,"SW7,SW15,SW596,SW407,SW1698,SW68,SW2,SW56,SW59...","W23420,W2610,W5506,W1489,W8749,W504,W1718,W252...",0,0,0,44,0,0,0,0,0,0,0,0,1725733


In [184]:
answer_df.to_csv(os.path.join(dataDir, 'answer_info_1021.csv'))

## 处理　invite info

In [189]:
invite_df = pd.read_csv(os.path.join(dataDir, 'invite_info_1020.csv'))

In [190]:
invite_df.head()

,question_id,user_id,create_time,is_answer
0,Q2166419046,M401693808,D3865-H22,0
1,Q1550017551,M3392373099,D3844-H11,0
2,Q604029601,M2317670257,D3862-H15,0
3,Q2350061229,M1618461867,D3849-H11,0
4,Q2443223942,M3544409350,D3867-H4,0


In [191]:
invite_df['create_day'] = invite_df['create_time'].apply(lambda x: int(x.split('-')[0].lstrip('D')))

In [192]:
invite_df['create_hour'] = invite_df['create_time'].apply(lambda x: int(x.split('-')[1].lstrip('H')))

In [194]:
invite_df.to_csv(os.path.join(dataDir, 'invite_info_1021.csv'),
                columns = ['question_id', 'user_id', 'create_time', 'create_day', 'create_hour', 'isAnswer'])